#  This Jupyter Notebook contains the codes of various Euler Bernoulli Beam simulations


## Constants definitions

In [2]:
L = 10; % length = 10 m
b = 0.1; % width = 10 cm
d = 0.05; % height = 5 cm
E = 2*10^11; % Young�s modulus for steel = 200 GPa = 2x10^11 Pa
I = b*d^3/12; % second moment of inertia
rho = 7850; % mass density of steel = 7850 kg/m^3
g = 9.81; % acceleration due to gravity = 9.81 m/s^2
w = rho*b*d*g; % weight of the beam per unit length (will be our f)

## Left fixed right free

In [16]:
% Since in Sauer's second edition does not use a pentadiagonal matrix, we must use a more standard approach
% Filling and solving our matrix.
format long
A = [];

n = 10; % number of subintervals on [0, L]
h = L/n; % discretization spacing
N = n + 1; % number of unknowns to be solved for

% Define the RHS of the system
f = -h^4/(E*I) * w * ones(N, 1);
f(1) = f(N) = 0;

% left end point
A(1,1) = 12;
A(1,2) = -9;
A(1,3) = (8/3);
A(1,4) = (-1/4);

% right endpoint
A(N,N) = (72/17);
A(N-1,N) = (-28/17);
A(N,N-1) = (-156/17);
A(N-1,N-1) = (72/17);
A(N-1,N-2) = (-60/17);
A(N,N-2) = (96/17);
A(N,N-3) = (-12/17);
A(N-1,N-3) = (16/17);


% Creating diagonals of the matrix
for i=1:N
    for j = 1:N
        % adding the main diagonal
        if i == j && i > 1 && i <= N - 2
            A(i,j) = 6;
        endif
        % adding the first superdiagonal and subdiagonal
        if abs(i - j) == 1 && i > 1 && i <= N - 2
            A(i,j) = -4;
        endif
        % adding the second superdiagonal and subdiagonal
        if abs(i - j) == 2 && i > 1 && i <= N - 2
            A(i,j) = 1;
        endif
    endfor
endfor


A
% Compressing the matrix into a sparse matrix
A = sparse(A);

% Finding y
y = A\f;

% Plots
x = (0:h:L)';
y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
ErrMax = max(abs(y-y_exact))

A =

 Columns 1 through 3:

   12.000000000000000   -9.000000000000000    2.666666666666667
   -4.000000000000000    6.000000000000000   -4.000000000000000
    1.000000000000000   -4.000000000000000    6.000000000000000
    0.000000000000000    1.000000000000000   -4.000000000000000
    0.000000000000000    0.000000000000000    1.000000000000000
    0.000000000000000    0.000000000000000    0.000000000000000
    0.000000000000000    0.000000000000000    0.000000000000000
    0.000000000000000    0.000000000000000    0.000000000000000
    0.000000000000000    0.000000000000000    0.000000000000000
    0.000000000000000    0.000000000000000    0.000000000000000
    0.000000000000000    0.000000000000000    0.000000000000000

 Columns 4 through 6:

   -0.250000000000000    0.000000000000000    0.000000000000000
    1.000000000000000    0.000000000000000    0.000000000000000
   -4.000000000000000    1.000000000000000    0.000000000000000
    6.000000000000000   -4.000000000000000    1.0000

In [ ]:
subplot(2,1,1);
plot(x, y, "b*", x, y_exact, "r")
title('Deflection of the beam');
xlabel('x'); ylabel('y (deflection)');
grid on;
subplot(2,1,2);
plot(x, abs(y - y_exact), "r")
title('Error: |computed - exact|');
xlabel('x'); ylabel('Error');
grid on;

In [2]:

% vector that contains error values
ErrorsBothFixed = [];
n = 10; % number of subintervals on [0, L]
h = L/n; % discretization spacing
N = n + 1;

for k=1:100 
    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);
    f(1) = f(N) = 0;
    % Since in Sauer's second edition does not use a pentadiagonal matrix, we must use a more standard approach
    % Filling and solving our matrix.
    format long
    A = [];

    % left end point
    A(1,1) = 12;
    A(1,2) = -9;
    A(1,3) = (8/3);
    A(1,4) = (-1/4);


    % right endpoint
    A(N,N) = (72/17);
    A(N-1,N) = (-28/17);
    A(N,N-1) = (-156/17);
    A(N-1,N-1) = (72/17);
    A(N-1,N-2) = (-60/17);
    A(N,N-2) = (96/17);
    A(N,N-3) = (-12/17);
    A(N-1,N-3) = (16/17);


    % Creating diagonals of the matrix
    for i=1:N
        for j = 1:N
            % adding the main diagonal
            if i == j && i > 1 && i <= N - 2
                A(i,j) = 6;
            endif
            % adding the first superdiagonal and subdiagonal
            if abs(i - j) == 1 && i > 1 && i <= N - 2
                A(i,j) = -4;
            endif
            % adding the second superdiagonal and subdiagonal
            if abs(i - j) == 2 && i > 1 && i <= N - 2
                A(i,j) = 1;
            endif
        endfor
    endfor


    % Compressing the matrix into a sparse matrix
    A = sparse(A);

    % Finding y
    y = A\f;

    % Plots

    x = (0:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrorsBothFixed(k) = max(abs(y-y_exact));
    n = n + 10;
    h = L/n; % discretization spacing
    N = n + 1;     
endfor

ErrorsBothFixed = ErrorsBothFixed'


ErrorsBothFixed =

   0.0462224014808181
   0.0453624415886269
   0.0446685534579458
   0.0443366272976597
   0.0440736860208084
   0.0438645407551958
   0.0437391934758395
   0.0436295778889470
   0.0435337432538501
   0.0434702920497669
   0.0434112506872436
   0.0433568023052379
   0.0433187808301543
   0.0432820539239362
   0.0432470576651339
   0.0432217936069488
   0.0431967878994157
   0.0431724716073767
   0.0431544472639346
   0.0431363393662978
   0.0431184727317580
   0.0431049755318861
   0.0430912634441011
   0.0430775856050199
   0.0430671042651655
   0.0430563633642498
   0.0430455580122629
   0.0430371852309180
   0.0430285455839958
   0.0430197949562812
   0.0430129536098729
   0.0430058542919543
   0.0429986238111732
   0.0429929295040163
   0.0429869927230732
   0.0429809182872517
   0.0429761051852416
   0.0429710672411866
   0.0429658923316371
   0.0429617707626188
   0.0429574420429198
   0.0429529807568647
   0.0429494117951763
   0.0429456524777022
   0.04294176